# Taylor Polynomial Approximations

Adam Rumpf

Created 4/15/21

Based on a [Mathematica class demonstration](https://github.com/adam-rumpf/mathematica-class-demonstrations#taylor-and-fourier-series-approximations).

See more Jupyter Notebook class demonstrations [here](https://github.com/adam-rumpf/jupyter-class-demonstrations).

## Motivation for Polynomial Approximation

In computational mathematics it's often useful to approximate a complicated function using a simpler one. This could be because the real function we're interested in is computationally expensive to evaluate, or because it leads to overly-complicated analytical results, or because it's actually unknown (as is the case in solving differential equations). However, as long as we have a bit of information about our function for one particular input, we may be able to reasonably approximate its value near that input.

Specifically, suppose we have a function $f(x)$ that we would like to approximate around some input value $a$. Our goal is to find a "simple" function $p(x)$ that is "approximately equal" to $f(x)$ for values of $x$ "near" $a$. That is,

\begin{align*}
\text{Find the polynomial $p(x)$ that is as close as possible to $f(x)$ near $a$.}
\end{align*}

There are lots of reasonable choices for what kind of function we could choose $p$ to be, but we happen to get some particularly nice results when we look at the case of $p$ being a polynomial.

In [1]:
# Pictures of polynomial approximations

## Building Up a Polynomial Approximation

Intuitively our goal is to define a polynomial $p(x)$ that is as close as possible to $f(x)$ near $a$. How exactly we do that depends on the degree of the polynomial, so let's go through the process of building up a sequence of increasingly-complicated degree-$n$ approximations $p_n$ to see whether we find any patterns that can be generalized.

### Constant Approximation

What degree-zero polynomial $p_0(x) = c_0$ does the best job of approximating $f(x)$ near $a$?

Well, a degree-zero polynomial is constant, so our only real job is to choose which constant value $c_0$ we want $p_0(x)$ to have. Since we've only been told to care about the input value $a$, the only thing we can really do is to guarantee that $p_0$ approximates $f$ exactly when $x=a$ by setting $c_0 = f(a)$. Then we get

\begin{align*}
p_0(x) &= f(a)
\end{align*}

In [2]:
### Interactive constant approximations

This is an okay approximation as long as the function value doesn't change very quickly around $a$, but otherwise $f(x)$ quickly diverges from $p_0(x)$ as $x$ moves further from $a$. However, we can try to fix that by using a more complicated polynomial approximation that takes this rate of change into account.

### Linear Approximation

What degree-one polynomial $p_1(x) = c_0 + c_1 x$ does the best job of approximating $f(x)$ near $a$?

A degree-one polynomial is linear, which gives us one additional coefficient to manipulate: Not only can we control the value of $p_1(x)$ and $a$, we can also control its _slope_. We certainly want to keep the property that $p_1(a) = f(a)$, since that means that $p_1(x)$ is as close as possible to $f(x)$ when $x$ is exactly equal to $a$, but what slope should we choose to make $p_1(x)$ as close to $f(x)$ as possible as we move away from $a$?

In [3]:
### Interactive slope manipulation

Intuitively we would like to choose the slope of $p_1(x)$ to match the slope of $f(x)$ as closely as possible near $a$, which we can do by setting the slope of $p_1(x)$ to equal the derivative of $f$ at $a$, or $f'(a)$.

Since we know that $p_1(x)$ should be a line passing through point $(a,f(a))$ with slope $f'(a)$, we can apply the point-slope formula for a line to arrive at the formula

\begin{align*}
p_1(x) &= f(a) + f'(a) \, (x-a)
\end{align*}

(Alternatively, we could arrive at this by beginning with the special case of $a=0$, in which case $(a,f(a)) = (0,f(0))$ is the same as the $y$-intercept and we can apply the slope-intercept formula to get $p_1(x) = f(a) + f'(a) \, x$. To generalize this to any $a$ we simply need to horizontally shift the $a=0$ graph by $a$ units by replacing $x$ with $x-a$, which gives the general linearization above.)

This is known as the _linear approximation_ of $f$ at $a$, and it appears as an essential part of many advanced numerical algorithms (such as the [Newton-Raphson method](https://en.wikipedia.org/wiki/Newton%27s_method) for root finding and [Euler's method](https://en.wikipedia.org/wiki/Euler_method) for numerical differential equation solution). Geometrically it is equivalent to finding the tangent line of $f$ at $a$.

In [4]:
### Interactive linear approximation

This is an okay approximation as long as the _slope_ of the function doesn't change very quickly around $a$, but if $f(x)$ curves very sharply then the linear approximation does not remain reasonable as $x$ moves away from $a$. So our next goal is to take this curvature into account.

### Quadratic Approximation

What degree-two polynomial $p_2(x) = c_0 + c_1 x + c_2 x^2$ does the best job of approximating $f(x)$ near $a$?

Since the previous two approximations consisted of ensuring that our polynomial would have the same function value as $f$ at $a$ (i.e. $p(a) = f(a)$) and the same slope as $f$ at $a$ (i.e. $p'(a) = f'(a)$), a reasonable next goal might be to ensure the correct _curvature_ as $f$ at $a$ (i.e. $p''(a) = f''(a)$). Looking at the previous two approximations, we might notice that the degree-zero approximation appears within the degree-one approximation,

\begin{align*}
p_0(x) &= f(a) \\
p_1(x) &= f(a) + f'(a) \, (x-a)
\end{align*}

This might cause us to wonder whether the solution could be as simple as just adding a third term to the polynomial of the form $c_2 (x-a)^2$, similar to how the previous approximation consisted of adding a second term of the form $c_1 (x-a)$. If such a thing were possible, we would want to choose $c_2$ in order to ensure that we maintain our previous two properties (that $p_2(a) = f(a)$ and $p'_2(a) = f'(a)$), but now also that $p''_2(a) = f''(a)$.

In [5]:
### Interactive quadratic approximation to play with the quadratic coefficient

This goal helps to explain why adding a third term of the form $c_2 (x-a)^2$ will help to give us the desired properties. The function value of $p_2$ at $a$ is

\begin{align*}
p_2(x) &= f(a) + f'(a) \, (x-a) + c_2 \, (x-a)^2 \\
\implies p_2(a) &= f(a) + f'(a) \, (a-a) + c_2 \, (a-a)^2 \\
&= f(a)
\end{align*}

which is the same function value as $f$ at $a$, as desired. The factor of $(x-a)$ in all non-constant terms vanishes when $x=a$, leaving only the constant term $f(a)$. Therefore we see that the constant term $f(a)$ is solely responsible for the function value of $p_2$ at $a$.

Likewise, the first derivative of $p_2$ at $a$ is

\begin{align*}
p'_2(x) &= f'(a) + 2c_2 \, (x-a) \\
\implies p'_2(a) &= f'(a) + 2c_2 \, (a-a) \\
&= f'(a)
\end{align*}

which is the same first derivative as $f$ at $a$, also as desired. The constant term $f(a)$ disappears when evaluating the first derivative and the quadratic term retains a factor of $(x-a)$ which vanishes when $x=a$, this time leaving only the coefficient of the linear term $f'(a)$. Therefore the linear term $f'(a)$ is solely responsible for the first derivative of $p_2$ at $a$.

Now to choose the quadratic coefficient $c_2$ to ensure that $p_2$ has the same second derivative as $f$ at $a$. Evaluating the second derivative, we are left with simply

\begin{align*}
p''_2(x) &= 2c_2
\end{align*}

so in order to have $p''_2(a) = f''(a)$ we can simply choose $c_2 = \frac{1}{2} f''(a)$. In this case the quadratic term $\frac{1}{2} f''(a)$ is solely responsible for the second derivative of $p_2$ at $a$, but we need to be careful to apply a factor of $\frac{1}{2}$ to account for the power rule being applied to $(x-a)^2$ in evaluating the second derivative of $p_2$. The full quadratic approximation is then

\begin{align*}
p_2(x) = f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2
\end{align*}

In [6]:
### Interactive quadratic approximations

In developing this most recent approximation we came upon an extremely useful property: If we set up our polynomial approximation in the right way, we can ensure that every term except for one vanishes when we evaluate a derivative of the polynomial at $a$, which then allows us to select each coefficient of the polynomial to independently define one of the derivatives of the polynomial at $a$. This process can be easily generalized to obtain a degree-$n$ polynomial whose first $n$ derivatives ($0$ through $n-1$) all match those of $f$ at $a$, since each new derivative can be matched by simply adding one new term to the polynomial without needing to change any of the previous ones.

However, we also saw in the previous example that some of these coefficients require an additional factor, so it might be helpful to go through the next couple of approximations by hand to work out the general form.

### Cubic Approximations

What degree-three polynomial $p_3(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3$ does the best job of approximating $f(x)$ near $a$?

Picking up from the previous example, we might imagine that the next approximation has the form

\begin{align*}
p_3(x) &= f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2 + c_3 \, (x-a)^3
\end{align*}

From the previous example we know that this polynomial satisfies the properties $p_3(a) = f(a)$, $p'_3(a) = f'(a)$, and $p''_3(a) = f''(a)$, since the lone new term includes a factor of $(x-a)$ which vanishes at $x=a$ in the zeroth, first, and second derivatives of $p_3$. By the time we evaluate the third derivative, all previous terms have vanished, leaving only

\begin{align*}
p'''_3(x) &= 3 \cdot 2 \cdot c_3
\end{align*}

with the factors of $3$ and $2$ left over from the first two applications of the power rule to $(x-a)^3$. In order to have $p'''_3(a) = f'''(a)$ we should then choose $c_3 = \frac{1}{6} f'''(a)$, for a cubic approximation of

\begin{align*}
p_3(x) = f(a) + f'(a) \, (x-a) + \frac{1}{2} f''(a) \, (x-a)^2 + \frac{1}{6} f'''(a) \, (x-a)^3
\end{align*}

In [7]:
### Playing with cubic approximations

### Quartic Approximations

What degree-four polynomial $p_4(x) = c_0 + c_1 x + c_2 x^2 + c_3 x^3 + c_4 x^4$ does the best job of approximating $f(x)$ near $a$?

Going through a similar process as the above, we find that the new term must be $\frac{1}{24} f^{(4)}(a) \, (x-a)^4$, with the factor of $\frac{1}{24}$ to compensate for applying the power rule to $(x-a)^4$ four times. This pattern can be seen more easily if we write $\frac{1}{4!}$ in place of $\frac{1}{24}$, partly for generalization, and partly to make it easier to see that the division by $4! = 4 \cdot 3 \cdot 2 \cdot 1$ compensates for the four applications of the power rule. Then the quartic approximation is

\begin{align*}
p_4(x) &= f(a) + f'(a) \, (x-a) + \frac{f''(a)}{2} (x-a)^2 + \frac{f'''(a)}{3!} (x-a)^3 + \frac{f^{(4)}(a)}{4!} (x-a)^4
\end{align*}

### Generalizing Polynomial Approximations

From our previous work we can see that adding more, higher-degree terms of a certain form allows us to match more, higher-order derivatives of $f$ at $a$. Intuitively this allows the polynomial approximation to match $f$ more closely over a larger range of inputs since it allows the polynomial to compensate for changes in the lower-order derivatives (matching the slope allows us to account for a rapidly-changing function value, matching curvature allows us to account for a rapidly-changing slope, etc.). The general form of the degree-$n$ term is

\begin{align*}
\frac{f^{(n)}(a)}{n!} (x-a)^n
\end{align*}

with a factor of $\frac{1}{n!}$ to compensate for $n$ applications of the power rule to $(x-a)^n$. This term is solely responsible for defining the $n$th derivative of $p_n(x)$, and ensures that $p_n^{(n)}(a) = f^{(a)}(a)$.

Written using summation notation, the degree-$n$ polynomial approximation of $f$ at $a$ is

\begin{align*}
p_n(x) = \sum_{m=0}^n \frac{f^{(m)}(a)}{m!} (x-a)^m
\end{align*}

Polynomial approximations of this form are called [Taylor polynomials](https://en.wikipedia.org/wiki/Taylor_series), and are used extensively throughout numerical analysis. In addition to the approximation applications discussed above, Taylor polynomials are often used in deriving error bounds and convergence criteria for numerical methods, since there are good theoertical bounds that can be placed on how close $p_n(x)$ is to $f(x)$.

In [8]:
### Playing with higher-order Taylor polynomials

While we set out with the goal of finding a "simple" function (a polynomial) that would be "close" to $f$ "near" $a$, one of the most surprising results about Taylor polynomials is that, for many functions, they can actually be extended to include infinitely many terms, at which point they cease to simply approximate $f$ near $a$ and actually become exactly equal to $f$ over the entire real line.

## Taylor Series

* (Showing Taylor series for some famous functions. Derive a few with specific examples, like the trigonometric functions and the exponential function.)
* (Show Euler's identity.)
* (Briefly discuss intervals of convergence.)

In [11]:
%matplotlib widget
#import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

# Define function derivatives
def f0(x):
    return np.sin(x)
def f1(x):
    return np.cos(x)
def f2(x):
    return -np.sin(x)
def f3(x):
    return -np.cos(x)
def g0(x):
    return np.exp(x)

# Define function lists
func = [[f0, f1, f2, f3]*2, [f1, f2, f3, f0]*2, [g0]*8]
fname = ["sin(x)", "cos(x)", "exp(x)"]

# Define Taylor polynomial
def taylor(x, fid, a, n):
    """Taylor polynomial for a given function.
    
    Positional arguments:
    x - input value
    fid - function ID from 'func' list
    a - center
    n - polynomial degree
    """
    
    out = 0.0 # output value
    
    # Add terms of Taylor polynomial
    for i in range(n+1):
        out += (func[fid][i](a)/np.math.factorial(i))*(x - a)**i
    
    return out

# Set up plot
fig, ax = plt.subplots()

# Generate x values
x = np.linspace(-2*np.pi, 2*np.pi, 101)
y = np.zeros_like(x) # initialize Taylor output array

# Draw plot lines
#@widgets.interact(a=(-2*np.pi, 2*np.pi, 0.05), f=(0, 2, 1), n=(0, 7, 1))
#def update1(a=0.0, f=0, n=1):
#    global ax, y
#    ax.clear()
#    ax.set_ylim([-4, 4])
#    plt.title(fname[f])
#    ax.grid(False)
#    ax.set_xlabel("x")
#    ax.set_ylabel("y")
#    ax.plot(x, func[f][0](x), color="C0")
#    y = taylor(x, f, a, n)
#    ax.plot(x, y, color="C1")
#    ax.plot(a, func[f][0](a), color="C1", marker=".", markersize=10)
ax.set_ylim([-4, 4])
plt.title(fname[0])
ax.grid(False)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.plot(x, func[0][0](x), color="C0")
y = taylor(x, 0, 0.0, 1)
ax.plot(x, y, color="C1")
ax.plot(0.0, func[0][0](0.0), color="C1", marker=".", markersize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …